## HUFF MODEL SIMULATION
* Attractiveness Factor: Multiply the attractiveness of the destination by some constant.
* Accessibility Factor: Divide by some function of the distance between the origin and destination.
* Alpha Parameter: Alpha is typically a parameter that adjusts the sensitivity of the model to distance.

In [1]:
# Library for data manipulation and analysis
import pandas as pd

# Library for working with ArcGIS tools and functionalities
import arcpy
import arcgis

# Libraries for working with PostgreSQL
import psycopg2
from psycopg2 import sql

# Library for handling JSON data
import json

# Library for making HTTP requests and interacting with web services
import requests

# Library for interacting with the operating system, managing file paths, and executing system commands
import os

# Library for handling warnings generated during code execution
import warnings

# Library for displaying images in Jupyter Notebooks
from IPython.display import Image

# Library for generating random numbers
import random

# Library for handling zip files and working with IO streams
import zipfile
import io

# Library for working with dates
from datetime import date

import urllib.request

### Path to FGDB And SDE

In [2]:
#path to local database
file_gdb = r"C:\Users\15612\Documents\Arc_II\Final_Project\BMSB_Project_2\BMSB_Project_2.gdb"

### Calculating Transition Probability  

In [3]:
# Define paths to tables and feature classes
City_Distance_Table = os.path.join(file_gdb, "MN_Cities_Distance")
BMSB_PointsData_SJ = os.path.join(file_gdb, "BMSB_PointsData_SJ")
MN_Cities_Pts = os.path.join(file_gdb, "MN_Cities_Pts")

In [18]:
# Function to calculate Huff model
def calculate_huff(fc, fields, alpha, scalar):
    # Initialize denominator for Huff model
    huff_denominator = 0
    
    # Search cursor to iterate through feature class
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        for row in cursor:
            city1_pop = row[3]
            city2_pop = row[4]
            near_dist = row[2]
            
            # Calculate numerator for Huff model
            huff_numerator = (city1_pop * city2_pop) / (near_dist ** alpha)
            
            # Add numerator to denominator
            huff_denominator += huff_numerator
            
        # Update cursor to calculate and assign probability of transfer for city pairs
        with arcpy.da.UpdateCursor(fc, fields) as cursor:
            for row in cursor:
                city1_pop = row[3]
                city2_pop = row[4]
                near_dist = row[2]
        
                # Calculate Huff value and scale it
                huff_numerator = (city1_pop * city2_pop) / (near_dist ** alpha)
                row[5] = scalar * (huff_numerator / huff_denominator)
                cursor.updateRow(row)

In [19]:
# Main code
fc = 'MN_Cities_Distance'
fields = ['City_1', 'City_2', 'Near_Dist', 'City_1_POP', 'City_2_POP', 'Trans_Prob_15']
alpha = 1.5
scalar = 30

# Call the function to calculate Huff model
calculate_huff(fc, fields, alpha, scalar)
print('Huff model calculation done.')

Huff model calculation done.


In [20]:
# Main code
fc = 'MN_Cities_Distance'
fields = ['City_1', 'City_2', 'Near_Dist', 'City_1_POP', 'City_2_POP', 'Trans_Prob_175']
alpha = 1.75
scalar = 30

# Call the function to calculate Huff model
calculate_huff(fc, fields, alpha, scalar)
print('Huff model calculation done.')

Huff model calculation done.


In [21]:
# Main code
fc = 'MN_Cities_Distance'
fields = ['City_1', 'City_2', 'Near_Dist', 'City_1_POP', 'City_2_POP', 'Trans_Prob_2']
alpha = 2
scalar = 30

# Call the function to calculate Huff model
calculate_huff(fc, fields, alpha, scalar)
print('Huff model calculation done.')

Huff model calculation done.


## Simulating the Spread of BMSB
* We are using Simulation Loop to iterate over each simulation from 0 to num_simulations - 1 (100 in this case)
* Our seed or starting city is Minneapolis - Results chnage if we start with St. Paul or any other city with high reported BMSB sightings 
* We are using 30 simulation steps, basically 30 discrete time steps or iterations
* We are using Presence and Absence Lists to retrieve cities with BMSB presence and absence 
* We are using Update Spread function to update spread in every single timestep

In [4]:
def simulateSpread(city_fc, huff_fc, city_fields, huff_fields, num_simulations, starting_city, timestep, file_gdb):
    
    # Iterate over the number of simulations
    for i in range(num_simulations):
        SimCol = 'BMSB_Sim' + str(i)
        
        # Populate simulation column with seed city
        city_fields.append(SimCol)

        with arcpy.da.UpdateCursor(city_fc, city_fields) as cursor:
            for row in cursor:
                if row[0] == starting_city:
                    row[2] = 1
                else:
                    row[2] = 0
                cursor.updateRow(row)

        # Perform spread simulation for each timestep
        for k in range(timestep):
            presence_list = []
            absence_list = []

            # Search cursor to iterate through feature class
            with arcpy.da.SearchCursor(city_fc, city_fields) as cursor:
                for row in cursor:
                    # Check if city is present or absent
                    if row[2] == 1:
                        presence_list.append(row[0])
                    else:
                        absence_list.append(row[0])
            
            spread_list = [] 
    
            # Search cursor to iterate through feature class
            with arcpy.da.SearchCursor(huff_fc, huff_fields) as cursor:
                for row in cursor:
                    # Check if cities in the presence list
                    if row[0] in presence_list:
                        dest_city = row[1]
                    elif row[1] in presence_list:
                        dest_city = row[0]
                    else:
                        continue

                    trans_prob = row[2]

                    # Spread to cities in the absence list based on transition probability
                    if dest_city in absence_list:
                        if trans_prob >= random.random():
                            spread_list.append(dest_city)
            
            # Update spread in a single timestep
            with arcpy.da.UpdateCursor(city_fc, city_fields) as cursor:
                for row in cursor:
                    if row[0] in spread_list:
                        row[2] = 1
                        cursor.updateRow(row)
                        
            print("Simulation " + str(i) + " | Timestep " + str(k) + ": BMSB spread to " + str(len(spread_list)) + " location(s): " + str(spread_list))
                        
        city_fields.remove(SimCol)
            
        print('Simulation', i, 'Complete')

    print("All Simulations Complete")

In [ ]:
simulateSpread(
    city_fc = 'MN_Cities_15', 
    huff_fc = 'MN_Cities_Distance', 
    city_fields = ['City','SUM_Population'], 
    huff_fields = ['City_1','City_2','Trans_Prob_15'], 
    num_simulations = 100, 
    starting_city = 'Falcon Heights', 
    timestep = 30, 
    file_gdb = file_gdb
)

Simulation 0 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 1: BMSB spread to 1 location(s): ['Minneapolis']
Simulation 0 | Timestep 2: BMSB spread to 7 location(s): ['Bloomington', 'Brooklyn Park', 'Edina', 'Robbinsdale', 'Saint Louis Park', 'Saint Paul', 'Shakopee']
Simulation 0 | Timestep 3: BMSB spread to 19 location(s): ['Albert Lea', 'Apple Valley', 'Eden Prairie', 'Maple Grove', 'New Brighton', 'Otsego', 'Crystal', 'Eden Prairie', 'Golden Valley', 'Inver Grove Heights', 'Lino Lakes', 'Lino Lakes', 'Maplewood', 'Medina', 'Roseville', 'Shoreview', 'New Brighton', 'Saint James', 'West Saint Paul']
Simulation 0 | Timestep 4: BMSB spread to 13 location(s): ['Blaine', 'Minnetonka', 'Plymouth', 'Champlin', 'Columbia Heights', 'North Saint Paul', 'Fridley', 'Lakeville', 'Lakeville', 'Richfield', 'Rosemount', 'White Bear Lake', 'Vadnais Heights']
Simulation 0 | Timestep 5: BMSB spread to 17 location(s): ['Lindstrom', 'Victoria', 'Ham Lake', 'Mounds View', 'Saint M

Simulation 4 | Timestep 18: BMSB spread to 8 location(s): ['Becker', 'Dayton', 'Grand Rapids', 'Kenyon', 'Madison Lake', 'Mahtomedi', 'Medina', 'North Saint Paul']
Simulation 4 | Timestep 19: BMSB spread to 4 location(s): ['Prior Lake', 'East Bethel', 'Victoria', 'Prior Lake']
Simulation 4 | Timestep 20: BMSB spread to 3 location(s): ['Albertville', 'Lake Crystal', 'Spring Lake Park']
Simulation 4 | Timestep 21: BMSB spread to 3 location(s): ['Credit River', 'Duluth', 'Pine Springs']
Simulation 4 | Timestep 22: BMSB spread to 3 location(s): ['Baxter', 'Nowthen', 'Winona']
Simulation 4 | Timestep 23: BMSB spread to 0 location(s): []
Simulation 4 | Timestep 24: BMSB spread to 3 location(s): ['Empire', 'Marshall', 'Saint Michael']
Simulation 4 | Timestep 25: BMSB spread to 3 location(s): ['Bayport', 'Circle Pines', 'Independence']
Simulation 4 | Timestep 26: BMSB spread to 5 location(s): ['Isanti', 'Willmar', 'Hermantown', 'Stillwater', 'Sunfish Lake']
Simulation 4 | Timestep 27: BMSB spr

Simulation 8 | Timestep 23: BMSB spread to 4 location(s): ['Lexington', 'Hoffman', 'Owatonna', 'Waite Park']
Simulation 8 | Timestep 24: BMSB spread to 1 location(s): ['Austin']
Simulation 8 | Timestep 25: BMSB spread to 2 location(s): ['Big Lake', 'Isanti']
Simulation 8 | Timestep 26: BMSB spread to 4 location(s): ['Canton', 'Excelsior', 'Kasson', 'North Oaks']
Simulation 8 | Timestep 27: BMSB spread to 4 location(s): ['Arlington', 'Zimmerman', 'Le Sueur', 'Lake Saint Croix Beach']
Simulation 8 | Timestep 28: BMSB spread to 3 location(s): ['Dayton', 'Eagle Lake', 'Saint Paul Park']
Simulation 8 | Timestep 29: BMSB spread to 2 location(s): ['Forest Lake', 'Credit River']
Simulation 8 Complete
Simulation 9 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 9 | Timestep 1: BMSB spread to 1 location(s): ['Saint Paul']
Simulation 9 | Timestep 2: BMSB spread to 4 location(s): ['Center City', 'Eagan', 'Maplewood', 'Minneapolis']
Simulation 9 | Timestep 3: BMSB spread to 16 location(s)

Simulation 13 | Timestep 4: BMSB spread to 17 location(s): ['Bloomington', 'Bloomington', 'Brooklyn Center', 'Columbia Heights', 'Columbia Heights', 'Coon Rapids', 'Lauderdale', 'Mendota Heights', 'Richfield', 'Robbinsdale', 'Saint Louis Park', 'Savage', 'Shoreview', 'Red Wing', 'Woodbury', 'Saint Peter', 'Woodbury']
Simulation 13 | Timestep 5: BMSB spread to 13 location(s): ['Apple Valley', 'Brooklyn Park', 'Burnsville', 'Crystal', 'Crystal', 'Eagan', 'Edina', 'Hugo', 'Lake Elmo', 'Orono', 'Medina', 'Rush City', 'Shakopee']
Simulation 13 | Timestep 6: BMSB spread to 12 location(s): ['Bayport', 'Elk River', 'Ham Lake', 'Rogers', 'Minnetonka', 'Dellwood', 'Duluth', 'Eden Prairie', 'Eden Prairie', 'Elk River', 'Lakeville', 'Owatonna']
Simulation 13 | Timestep 7: BMSB spread to 16 location(s): ['Andover', 'Blaine', 'Blaine', 'Blaine', 'Northfield', 'Chanhassen', 'South Saint Paul', 'Shorewood', 'New Hope', 'Farmington', 'Fridley', 'Ramsey', 'Plymouth', 'North Branch', 'Oakdale', 'Victoria

Simulation 17 | Timestep 7: BMSB spread to 13 location(s): ['Grant', 'Champlin', 'Champlin', 'Crystal', 'Crystal', 'Empire', 'Faribault', 'Saint Cloud', 'Mendota Heights', 'Otsego', 'Stillwater', 'White Bear Lake', 'White Bear Lake']
Simulation 17 | Timestep 8: BMSB spread to 3 location(s): ['Long Lake', 'Dayton', 'Rogers']
Simulation 17 | Timestep 9: BMSB spread to 8 location(s): ['Carver', 'Coon Rapids', 'Mapleton', 'Forest Lake', 'Golden Valley', 'Mound', 'Vadnais Heights', 'Newport']
Simulation 17 | Timestep 10: BMSB spread to 7 location(s): ['Cottage Grove', 'Wells', 'Hopkins', 'Hugo', 'North Mankato', 'North Oaks', 'Watertown']
Simulation 17 | Timestep 11: BMSB spread to 6 location(s): ['East Bethel', 'Hutchinson', 'Litchfield', 'Wayzata', 'Victoria', 'Waconia']
Simulation 17 | Timestep 12: BMSB spread to 3 location(s): ['Northfield', 'Shorewood', 'Lindstrom']
Simulation 17 | Timestep 13: BMSB spread to 5 location(s): ['Buffalo', 'Duluth', 'Elk River', 'Hanover', 'Hastings']
Simu

...

Simulation 81 | Timestep 23: BMSB spread to 5 location(s): ['Ramsey', 'Wyoming', 'Lilydale', 'Saint Bonifacius', 'Waconia']
Simulation 81 | Timestep 24: BMSB spread to 4 location(s): ['Rush City', 'Mahtomedi', 'Northfield', 'Saint Anthony']
Simulation 81 | Timestep 25: BMSB spread to 8 location(s): ['Belle Plaine', 'Excelsior', 'Deephaven', 'Red Wing', 'Lindstrom', 'Medford', 'Minnetrista', 'Waite Park']
Simulation 81 | Timestep 26: BMSB spread to 8 location(s): ['East Bethel', 'East Bethel', 'Rogers', 'Credit River', 'Mound', 'Spring Lake Park', 'Rogers', 'Spring Lake Park']
Simulation 81 | Timestep 27: BMSB spread to 2 location(s): ['Centerville', 'Norwood Young America']
Simulation 81 | Timestep 28: BMSB spread to 1 location(s): ['Victoria']
Simulation 81 | Timestep 29: BMSB spread to 6 location(s): ['Glencoe', 'Empire', 'Maple Plain', 'Miesville', 'Pine Island', 'Saint Paul Park']
Simulation 81 Complete
Simulation 82 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 82 | Ti

Simulation 86 | Timestep 1: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 2: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 3: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 4: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 5: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 6: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 7: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 8: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 9: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 10: BMSB spread to 1 location(s): ['Maple Grove']
Simulation 86 | Timestep 11: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 12: BMSB spread to 1 location(s): ['Minneapolis']
Simulation 86 | Timestep 13: BMSB spread to 9 location(s): ['Bloomington', 'Dayton', 'Inver Grove Heights', 'Lakeville', 'Lakeville', 'Newport', 'Plymouth', 'Roseville', 'Saint Paul']
Simulation 86 | Timestep 14: BMSB spread to 17 location

Simulation 90 | Timestep 5: BMSB spread to 11 location(s): ['Elk River', 'Saint Bonifacius', 'Redwood Falls', 'Circle Pines', 'Savage', 'Crystal', 'Deephaven', 'Edina', 'Inver Grove Heights', 'Oakdale', 'Northfield']
Simulation 90 | Timestep 6: BMSB spread to 6 location(s): ['Albertville', 'Saint Michael', 'Chanhassen', 'Chanhassen', 'Otsego', 'Prior Lake']
Simulation 90 | Timestep 7: BMSB spread to 11 location(s): ['Big Lake', 'Forest Lake', 'Fridley', 'Red Wing', 'Mounds View', 'East Gull Lake', 'Hayfield', 'Medina', 'Minnetonka', 'Minnetonka Beach', 'Princeton']
Simulation 90 | Timestep 8: BMSB spread to 7 location(s): ['Anoka', 'Apple Valley', 'Shakopee', 'Cottage Grove', 'Hopkins', 'Lauderdale', 'New Hope']
Simulation 90 | Timestep 9: BMSB spread to 11 location(s): ['Stillwater', 'Spring Lake Park', 'Duluth', 'East Bethel', 'Excelsior', 'South Saint Paul', 'White Bear Lake', 'White Bear Lake', 'North Branch', 'North Saint Paul', 'Oak Grove']
Simulation 90 | Timestep 10: BMSB sprea

Simulation 94 | Timestep 11: BMSB spread to 8 location(s): ['East Bethel', 'Rosemount', 'Litchfield', 'Mankato', 'Prior Lake', 'Newport', 'Zumbrota', 'South Saint Paul']
Simulation 94 | Timestep 12: BMSB spread to 5 location(s): ['Arden Hills', 'Faribault', 'Saint Anthony', 'Saint Joseph', 'Vadnais Heights']
Simulation 94 | Timestep 13: BMSB spread to 6 location(s): ['Maple Plain', 'North Branch', 'Buffalo', 'Cokato', 'Hastings', 'Stillwater']
Simulation 94 | Timestep 14: BMSB spread to 7 location(s): ['Circle Pines', 'Corcoran', 'Good Thunder', 'Inver Grove Heights', 'Long Beach', 'Red Wing', 'Shorewood']
Simulation 94 | Timestep 15: BMSB spread to 2 location(s): ['Afton', 'Lexington']
Simulation 94 | Timestep 16: BMSB spread to 4 location(s): ['Austin', 'Watertown', 'Kimball', 'Lauderdale']
Simulation 94 | Timestep 17: BMSB spread to 3 location(s): ['Dayton', 'Victoria', 'Jordan']
Simulation 94 | Timestep 18: BMSB spread to 4 location(s): ['Lake Elmo', 'Ramsey', 'North Oaks', 'Montic

Simulation 98 | Timestep 19: BMSB spread to 6 location(s): ['Big Lake', 'Chaska', 'Lonsdale', 'Spring Lake Park', 'Oakdale', 'Saint Joseph']
Simulation 98 | Timestep 20: BMSB spread to 8 location(s): ['Arlington', 'Lexington', 'Cambridge', 'Circle Pines', 'Park Rapids', 'Sartell', 'New London', 'Northfield']
Simulation 98 | Timestep 21: BMSB spread to 7 location(s): ['Andover', 'East Grand Forks', 'Rockford', 'Buffalo', 'Victoria', 'Oak Park Heights', 'Saint Cloud']
Simulation 98 | Timestep 22: BMSB spread to 2 location(s): ['Rogers', 'Wayzata']
Simulation 98 | Timestep 23: BMSB spread to 3 location(s): ['Anoka', 'Worthington', 'Mankato']
Simulation 98 | Timestep 24: BMSB spread to 3 location(s): ['Bigfork', 'Byron', 'Montrose']
Simulation 98 | Timestep 25: BMSB spread to 8 location(s): ['Forest Lake', 'Centerville', 'Corcoran', 'Dayton', 'Orono', 'Owatonna', 'Otsego', 'Saint Michael']
Simulation 98 | Timestep 26: BMSB spread to 5 location(s): ['Montgomery', 'Hilltop', 'Dundas', 'Wasec

In [ ]:
simulateSpread(
    city_fc = 'MN_Cities_175', 
    huff_fc = 'MN_Cities_Distance', 
    city_fields = ['City','SUM_Population'], 
    huff_fields = ['City_1','City_2','Trans_Prob_175'], 
    num_simulations = 100, 
    starting_city = 'Falcon Heights', 
    timestep = 30, 
    file_gdb = file_gdb
)

Simulation 0 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 1: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 2: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 3: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 4: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 5: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 6: BMSB spread to 1 location(s): ['Minneapolis']
Simulation 0 | Timestep 7: BMSB spread to 11 location(s): ['Deephaven', 'Delano', 'Eden Prairie', 'Edina', 'Fridley', 'Jordan', 'Mendota Heights', 'Minnetonka', 'Plymouth', 'Robbinsdale', 'Saint Paul']
Simulation 0 | Timestep 8: BMSB spread to 14 location(s): ['Apple Valley', 'Bloomington', 'Brooklyn Park', 'Columbia Heights', 'Coon Rapids', 'Crystal', 'Eagan', 'Farmington', 'Lino Lakes', 'Saint Louis Park', 'Shakopee', 'Zimmerman', 'Otsego', 'Watertown']
Simulation 0 | Timestep 9: BMSB spread to 13 location(s): ['Anoka', 'Burnsville', 'Lakeville', 'Brooklyn Ce

Simulation 4 | Timestep 18: BMSB spread to 12 location(s): ['Andover', 'Saint Paul Park', 'Eagan', 'Eagan', 'Rosemount', 'Savage', 'Elk River', 'Little Canada', 'Minnetonka', 'Oakdale', 'Shakopee', 'Saint Cloud']
Simulation 4 | Timestep 19: BMSB spread to 15 location(s): ['Winnebago', 'Brooklyn Center', 'Brooklyn Park', 'Burnsville', 'Coon Rapids', 'Deephaven', 'Medina', 'Hopkins', 'Mahtomedi', 'Mendota Heights', 'Shorewood', 'Red Wing', 'Rochester', 'Rogers', 'Sartell']
Simulation 4 | Timestep 20: BMSB spread to 11 location(s): ['Blaine', 'Crystal', 'Corcoran', 'Crystal', 'Excelsior', 'Owatonna', 'Fridley', 'Litchfield', 'Lakeville', 'Orono', 'Wyoming']
Simulation 4 | Timestep 21: BMSB spread to 6 location(s): ['Watertown', 'Zimmerman', 'Mounds View', 'Oak Grove', 'Wayzata', 'Sauk Rapids']
Simulation 4 | Timestep 22: BMSB spread to 6 location(s): ['Anoka', 'Cambridge', 'Champlin', 'Hastings', 'Lauderdale', 'Mound']
Simulation 4 | Timestep 23: BMSB spread to 9 location(s): ['Stillwater

Simulation 9 | Timestep 16: BMSB spread to 13 location(s): ['Birchwood Village', 'Inver Grove Heights', 'Coon Rapids', 'Cottage Grove', 'New Brighton', 'Medicine Lake', 'New Hope', 'Plymouth', 'Richfield', 'Minnetonka', 'Oakdale', 'West Saint Paul', 'White Bear Lake']
Simulation 9 | Timestep 17: BMSB spread to 15 location(s): ['Apple Valley', 'Blaine', 'Bloomington', 'Bloomington', 'Shakopee', 'Eagan', 'Eden Prairie', 'Maple Grove', 'Mounds View', 'Landfall', 'Greenfield', 'Wayzata', 'Savage', 'North Oaks', 'Wayzata']
Simulation 9 | Timestep 18: BMSB spread to 12 location(s): ['Columbia Heights', 'Brooklyn Park', 'Brooklyn Park', 'Champlin', 'Chaska', 'Crystal', 'Hopkins', 'Lake Elmo', 'Little Canada', 'Maplewood', 'Maplewood', 'North Saint Paul']
Simulation 9 | Timestep 19: BMSB spread to 6 location(s): ['Brooklyn Center', 'Brooklyn Center', 'Chanhassen', 'Hugo', 'Vadnais Heights', 'Robbinsdale']
Simulation 9 | Timestep 20: BMSB spread to 7 location(s): ['Rosemount', 'Medina', 'Circle

Simulation 13 | Timestep 20: BMSB spread to 4 location(s): ['Albert Lea', 'Dayton', 'Oak Grove', 'Stewartville']
Simulation 13 | Timestep 21: BMSB spread to 3 location(s): ['Kimball', 'Zumbrota', 'Saint Francis']
Simulation 13 | Timestep 22: BMSB spread to 6 location(s): ['Annandale', 'Sauk Rapids', 'Osseo', 'Ham Lake', 'Woodland', 'Pipestone']
Simulation 13 | Timestep 23: BMSB spread to 4 location(s): ['Afton', 'Andover', 'Holdingford', 'Jordan']
Simulation 13 | Timestep 24: BMSB spread to 1 location(s): ['Hutchinson']
Simulation 13 | Timestep 25: BMSB spread to 4 location(s): ['Benson', 'Northfield', 'Northfield', 'Pine Springs']
Simulation 13 | Timestep 26: BMSB spread to 1 location(s): ['Maple Plain']
Simulation 13 | Timestep 27: BMSB spread to 6 location(s): ['East Bethel', 'Cambridge', 'Hayfield', 'Lexington', 'Orono', 'Owatonna']
Simulation 13 | Timestep 28: BMSB spread to 2 location(s): ['Long Lake', 'Spring Park']
Simulation 13 | Timestep 29: BMSB spread to 3 location(s): ['Ba

Simulation 18 | Timestep 5: BMSB spread to 6 location(s): ['Bloomington', 'Lakeville', 'Lexington', 'Minneapolis', 'Saint Louis Park', 'West Saint Paul']
Simulation 18 | Timestep 6: BMSB spread to 13 location(s): ['Maple Grove', 'Richfield', 'Brooklyn Park', 'Burnsville', 'East Bethel', 'Edina', 'Golden Valley', 'Mahtomedi', 'Maple Grove', 'Medina', 'Robbinsdale', 'New Brighton', 'Woodbury']
Simulation 18 | Timestep 7: BMSB spread to 10 location(s): ['Hugo', 'Cottage Grove', 'Dayton', 'Ham Lake', 'Plymouth', 'Plymouth', 'Saint Anthony', 'Shakopee', 'Minnetonka', 'New Ulm']
Simulation 18 | Timestep 8: BMSB spread to 14 location(s): ['Andover', 'Blaine', 'Brooklyn Center', 'Coon Rapids', 'Duluth', 'Eagan', 'Orono', 'Inver Grove Heights', 'Shorewood', 'Maplewood', 'Maplewood', 'Mendota Heights', 'Nowthen', 'Roseville']
Simulation 18 | Timestep 9: BMSB spread to 18 location(s): ['Anoka', 'Anoka', 'Apple Valley', 'Apple Valley', 'Arden Hills', 'North Oaks', 'Delano', 'Mounds View', 'Ramsey'

...

Simulation 77 | Timestep 29: BMSB spread to 5 location(s): ['Byron', 'Long Lake', 'Empire', 'Mankato', 'Saint Charles']
Simulation 77 Complete
Simulation 78 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 1: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 2: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 3: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 4: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 5: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 6: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 7: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 8: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 9: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 10: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 11: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 12: BMSB spread to 0 location(s): []
Simulation 78 | Timestep 13: BMSB spread to 1 location(s): ['

Simulation 82 | Timestep 13: BMSB spread to 3 location(s): ['Hibbing', 'Prior Lake', 'Rosemount']
Simulation 82 | Timestep 14: BMSB spread to 11 location(s): ['Columbus', 'Cottage Grove', 'Delano', 'Northfield', 'Farmington', 'Ham Lake', 'Hugo', 'Lilydale', 'Lino Lakes', 'Newport', 'Oakdale']
Simulation 82 | Timestep 15: BMSB spread to 6 location(s): ['Bovey', 'Monticello', 'Osseo', 'Isanti', 'Little Canada', 'Wyoming']
Simulation 82 | Timestep 16: BMSB spread to 3 location(s): ['Lauderdale', 'Chanhassen', 'Chaska']
Simulation 82 | Timestep 17: BMSB spread to 2 location(s): ['Duluth', 'Victoria']
Simulation 82 | Timestep 18: BMSB spread to 4 location(s): ['Andover', 'Andover', 'Centerville', 'Mankato']
Simulation 82 | Timestep 19: BMSB spread to 6 location(s): ['Afton', 'Nowthen', 'Faribault', 'East Bethel', 'Lake Elmo', 'Saint Michael']
Simulation 82 | Timestep 20: BMSB spread to 3 location(s): ['Belle Plaine', 'Saint Anthony', 'Vadnais Heights']
Simulation 82 | Timestep 21: BMSB spre

Simulation 86 | Timestep 28: BMSB spread to 3 location(s): ['Nowthen', 'North Mankato', 'Northfield']
Simulation 86 | Timestep 29: BMSB spread to 0 location(s): []
Simulation 86 Complete
Simulation 87 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 87 | Timestep 1: BMSB spread to 0 location(s): []
Simulation 87 | Timestep 2: BMSB spread to 0 location(s): []
Simulation 87 | Timestep 3: BMSB spread to 0 location(s): []
Simulation 87 | Timestep 4: BMSB spread to 1 location(s): ['Minneapolis']
Simulation 87 | Timestep 5: BMSB spread to 11 location(s): ['Blaine', 'Brooklyn Center', 'Brooklyn Park', 'Coon Rapids', 'Monticello', 'North Oaks', 'Plymouth', 'Saint Anthony', 'Saint Louis Park', 'Saint Paul', 'Vadnais Heights']
Simulation 87 | Timestep 6: BMSB spread to 19 location(s): ['Apple Valley', 'Bloomington', 'Carver', 'Chaska', 'Chaska', 'Crystal', 'Dellwood', 'Eagan', 'Edina', 'Golden Valley', 'Lakeville', 'New Hope', 'Oakdale', 'Richfield', 'Robbinsdale', 'New Hope', 'Savage',

Simulation 91 | Timestep 12: BMSB spread to 16 location(s): ['Andover', 'Shoreview', 'Burnsville', 'Farmington', 'Champlin', 'Plymouth', 'Roseville', 'Edina', 'Hopkins', 'Inver Grove Heights', 'Orono', 'Richfield', 'Saint Louis Park', 'Shoreview', 'West Saint Paul', 'Woodbury']
Simulation 91 | Timestep 13: BMSB spread to 13 location(s): ['Lakeville', 'Blaine', 'Blaine', 'Blaine', 'Lakeville', 'Eden Prairie', 'Elko New Market', 'Golden Valley', 'Maple Grove', 'New Hope', 'Wayzata', 'Saint Anthony', 'Saint Michael']
Simulation 91 | Timestep 14: BMSB spread to 13 location(s): ['Rosemount', 'Oakdale', 'Ramsey', 'Crystal', 'Minnetonka', 'Greenfield', 'Lake Elmo', 'Oakdale', 'Maplewood', 'Rogers', 'Savage', 'North Oaks', 'Oakdale']
Simulation 91 | Timestep 15: BMSB spread to 3 location(s): ['Ham Lake', 'Northfield', 'Vadnais Heights']
Simulation 91 | Timestep 16: BMSB spread to 12 location(s): ['Anoka', 'Coon Rapids', 'Fridley', 'Coon Rapids', 'Credit River', 'Excelsior', 'Robbinsdale', 'Lon

Simulation 95 | Timestep 17: BMSB spread to 2 location(s): ['Saint Francis', 'Lauderdale']
Simulation 95 | Timestep 18: BMSB spread to 5 location(s): ['Corcoran', 'East Bethel', 'Faribault', 'New Prague', 'Shorewood']
Simulation 95 | Timestep 19: BMSB spread to 1 location(s): ['Osseo']
Simulation 95 | Timestep 20: BMSB spread to 4 location(s): ['Alexandria', 'Saint Paul Park', 'Empire', 'Isanti']
Simulation 95 | Timestep 21: BMSB spread to 3 location(s): ['Lake Saint Croix Beach', 'Dodge Center', 'Waite Park']
Simulation 95 | Timestep 22: BMSB spread to 3 location(s): ['Centerville', 'Vadnais Heights', 'Oak Grove']
Simulation 95 | Timestep 23: BMSB spread to 2 location(s): ['Wyoming', 'Kasson']
Simulation 95 | Timestep 24: BMSB spread to 0 location(s): []
Simulation 95 | Timestep 25: BMSB spread to 1 location(s): ['Owatonna']
Simulation 95 | Timestep 26: BMSB spread to 5 location(s): ['Elko New Market', 'Columbus', 'Delano', 'Forest Lake', 'Forest Lake']
Simulation 95 | Timestep 27: BM

In [5]:
simulateSpread(
    city_fc = 'MN_Cities_2', 
    huff_fc = 'MN_Cities_Distance', 
    city_fields = ['City','SUM_Population'], 
    huff_fields = ['City_1','City_2','Trans_Prob_2'], 
    num_simulations = 100, 
    starting_city = 'Falcon Heights', 
    timestep = 30, 
    file_gdb = file_gdb
)

Simulation 0 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 1: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 2: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 3: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 4: BMSB spread to 0 location(s): []
Simulation 0 | Timestep 5: BMSB spread to 1 location(s): ['Saint Paul']
Simulation 0 | Timestep 6: BMSB spread to 9 location(s): ['Brooklyn Park', 'Edina', 'Minneapolis', 'Maplewood', 'Mendota Heights', 'Minneapolis', 'Ramsey', 'Roseville', 'West Saint Paul']
Simulation 0 | Timestep 7: BMSB spread to 12 location(s): ['Brooklyn Center', 'East Bethel', 'Coon Rapids', 'Eagan', 'Eden Prairie', 'Farmington', 'Golden Valley', 'Oakdale', 'Robbinsdale', 'Saint Louis Park', 'Minnetonka', 'North Saint Paul']
Simulation 0 | Timestep 8: BMSB spread to 14 location(s): ['Blaine', 'Blaine', 'Bloomington', 'Carver', 'Hugo', 'Burnsville', 'Chanhassen', 'Columbia Heights', 'Ham Lake', 'Maple Grove', 'Richfiel

Simulation 4 | Timestep 18: BMSB spread to 9 location(s): ['Austin', 'Big Lake', 'Circle Pines', 'Lauderdale', 'Independence', 'Landfall', 'Wayzata', 'Monticello', 'Rosemount']
Simulation 4 | Timestep 19: BMSB spread to 5 location(s): ['Afton', 'Birchwood Village', 'Centerville', 'Medina', 'Medina']
Simulation 4 | Timestep 20: BMSB spread to 0 location(s): []
Simulation 4 | Timestep 21: BMSB spread to 6 location(s): ['Lexington', 'Chaska', 'Hilltop', 'Lake Elmo', 'Lonsdale', 'Shorewood']
Simulation 4 | Timestep 22: BMSB spread to 0 location(s): []
Simulation 4 | Timestep 23: BMSB spread to 0 location(s): []
Simulation 4 | Timestep 24: BMSB spread to 1 location(s): ['Greenwood']
Simulation 4 | Timestep 25: BMSB spread to 5 location(s): ['Atwater', 'Carver', 'Grant', 'Hutchinson', 'Oak Park Heights']
Simulation 4 | Timestep 26: BMSB spread to 4 location(s): ['Albertville', 'Dodge Center', 'Faribault', 'Hastings']
Simulation 4 | Timestep 27: BMSB spread to 5 location(s): ['Orono', 'Montro

Simulation 9 | Timestep 8: BMSB spread to 11 location(s): ['Fridley', 'Plymouth', 'Fridley', 'Ramsey', 'Eagan', 'Eagan', 'Eagan', 'Oakdale', 'New Hope', 'Plymouth', 'South Saint Paul']
Simulation 9 | Timestep 9: BMSB spread to 13 location(s): ['Lino Lakes', 'North Oaks', 'Cottage Grove', 'Crystal', 'Dayton', 'Hugo', 'Hugo', 'Lakeland Shores', 'Newport', 'Newport', 'Oak Park Heights', 'Saint Cloud', 'White Bear Lake']
Simulation 9 | Timestep 10: BMSB spread to 8 location(s): ['Inver Grove Heights', 'Champlin', 'Osseo', 'Centerville', 'Spring Lake Park', 'Lake Elmo', 'North Saint Paul', 'Sauk Rapids']
Simulation 9 | Timestep 11: BMSB spread to 5 location(s): ['Farmington', 'Maple Grove', 'Maple Grove', 'Rosemount', 'Sartell']
Simulation 9 | Timestep 12: BMSB spread to 8 location(s): ['Lexington', 'Cokato', 'Mendota Heights', 'Glencoe', 'Lakeville', 'Vadnais Heights', 'Saint Paul Park', 'Vadnais Heights']
Simulation 9 | Timestep 13: BMSB spread to 2 location(s): ['Grant', 'Sunfish Lake']


Simulation 13 | Timestep 23: BMSB spread to 4 location(s): ['Dayton', 'Glencoe', 'Hastings', 'New Prague']
Simulation 13 | Timestep 24: BMSB spread to 4 location(s): ['Hanover', 'Waseca', 'Mounds View', 'Red Wing']
Simulation 13 | Timestep 25: BMSB spread to 2 location(s): ['Waite Park', 'Stillwater']
Simulation 13 | Timestep 26: BMSB spread to 4 location(s): ['Saint Francis', 'Hilltop', 'Empire', 'Lilydale']
Simulation 13 | Timestep 27: BMSB spread to 1 location(s): ['Rogers']
Simulation 13 | Timestep 28: BMSB spread to 1 location(s): ['Stacy']
Simulation 13 | Timestep 29: BMSB spread to 1 location(s): ['Excelsior']
Simulation 13 Complete
Simulation 14 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 14 | Timestep 1: BMSB spread to 0 location(s): []
Simulation 14 | Timestep 2: BMSB spread to 0 location(s): []
Simulation 14 | Timestep 3: BMSB spread to 0 location(s): []
Simulation 14 | Timestep 4: BMSB spread to 0 location(s): []
Simulation 14 | Timestep 5: BMSB spread to 0 lo

Simulation 18 | Timestep 6: BMSB spread to 11 location(s): ['Andover', 'Spring Lake Park', 'Chanhassen', 'Corcoran', 'Saint Marys Point', 'Savage', 'Lexington', 'Shakopee', 'Winona', 'Rosemount', 'Savage']
Simulation 18 | Timestep 7: BMSB spread to 6 location(s): ['Victoria', 'Hopkins', 'Hopkins', 'Landfall', 'Saint Anthony', 'Shoreview']
Simulation 18 | Timestep 8: BMSB spread to 5 location(s): ['Arden Hills', 'Robbinsdale', 'Robbinsdale', 'Newport', 'North Saint Paul']
Simulation 18 | Timestep 9: BMSB spread to 4 location(s): ['Anoka', 'Mounds View', 'Otsego', 'North Oaks']
Simulation 18 | Timestep 10: BMSB spread to 7 location(s): ['Saint Francis', 'Lake Saint Croix Beach', 'Ramsey', 'Credit River', 'Forest Lake', 'Hutchinson', 'Orono']
Simulation 18 | Timestep 11: BMSB spread to 4 location(s): ['Empire', 'Rogers', 'Montrose', 'Stillwater']
Simulation 18 | Timestep 12: BMSB spread to 2 location(s): ['Mendota Heights', 'Saint Bonifacius']
Simulation 18 | Timestep 13: BMSB spread to 3

...

Simulation 77 | Timestep 10: BMSB spread to 11 location(s): ['Anoka', 'Blaine', 'Champlin', 'Eagan', 'Mound', 'Lakeland', 'Northfield', 'Robbinsdale', 'Spring Lake Park', 'Shakopee', 'Robbinsdale']
Simulation 77 | Timestep 11: BMSB spread to 10 location(s): ['Albertville', 'Lexington', 'Burnsville', 'Chaska', 'Elk River', 'Kasson', 'Lino Lakes', 'Mounds View', 'Orono', 'Orono']
Simulation 77 | Timestep 12: BMSB spread to 5 location(s): ['Ramsey', 'Apple Valley', 'Lakeville', 'Sauk Centre', 'White Bear Lake']
Simulation 77 | Timestep 13: BMSB spread to 7 location(s): ['Corcoran', 'Rosemount', 'Independence', 'Elgin', 'Madelia', 'Newport', 'Otsego']
Simulation 77 | Timestep 14: BMSB spread to 9 location(s): ['Hilltop', 'Centerville', 'Hugo', 'Elko New Market', 'Farmington', 'Forest Lake', 'Grant', 'Lilydale', 'Shorewood']
Simulation 77 | Timestep 15: BMSB spread to 4 location(s): ['Buffalo', 'Oak Grove', 'Stillwater', 'Medina']
Simulation 77 | Timestep 16: BMSB spread to 2 location(s): [

Simulation 81 | Timestep 25: BMSB spread to 1 location(s): ['Big Lake']
Simulation 81 | Timestep 26: BMSB spread to 0 location(s): []
Simulation 81 | Timestep 27: BMSB spread to 4 location(s): ['Lexington', 'Cohasset', 'East Bethel', 'Minnetrista']
Simulation 81 | Timestep 28: BMSB spread to 0 location(s): []
Simulation 81 | Timestep 29: BMSB spread to 3 location(s): ['Isle', 'Deephaven', 'Spring Park']
Simulation 81 Complete
Simulation 82 | Timestep 0: BMSB spread to 1 location(s): ['Saint Paul']
Simulation 82 | Timestep 1: BMSB spread to 4 location(s): ['Minneapolis', 'Maplewood', 'Minneapolis', 'Roseville']
Simulation 82 | Timestep 2: BMSB spread to 8 location(s): ['Blaine', 'Golden Valley', 'Richfield', 'Robbinsdale', 'Saint Louis Park', 'Richfield', 'West Saint Paul', 'Wyoming']
Simulation 82 | Timestep 3: BMSB spread to 14 location(s): ['Bloomington', 'Bloomington', 'Columbia Heights', 'Coon Rapids', 'Edina', 'Forest Lake', 'Maple Grove', 'Jordan', 'Lino Lakes', 'Oakdale', 'Mound

Simulation 86 | Timestep 13: BMSB spread to 5 location(s): ['Otsego', 'Saint Anthony', 'White Bear Lake', 'Rogers', 'Winona']
Simulation 86 | Timestep 14: BMSB spread to 4 location(s): ['Credit River', 'Elk River', 'Elk River', 'South Saint Paul']
Simulation 86 | Timestep 15: BMSB spread to 3 location(s): ['Glencoe', 'Lauderdale', 'Spring Lake Park']
Simulation 86 | Timestep 16: BMSB spread to 0 location(s): []
Simulation 86 | Timestep 17: BMSB spread to 2 location(s): ['Mendota Heights', 'New Ulm']
Simulation 86 | Timestep 18: BMSB spread to 2 location(s): ['Medina', 'Sunfish Lake']
Simulation 86 | Timestep 19: BMSB spread to 5 location(s): ['Empire', 'Wayzata', 'Shorewood', 'Wayzata', 'Willernie']
Simulation 86 | Timestep 20: BMSB spread to 3 location(s): ['Lexington', 'Hugo', 'Stacy']
Simulation 86 | Timestep 21: BMSB spread to 1 location(s): ['Landfall']
Simulation 86 | Timestep 22: BMSB spread to 6 location(s): ['Ramsey', 'Cambridge', 'Independence', 'Columbus', 'Victoria', 'Indep

Simulation 91 | Timestep 7: BMSB spread to 2 location(s): ['Minneapolis', 'Roseville']
Simulation 91 | Timestep 8: BMSB spread to 8 location(s): ['Burnsville', 'Edina', 'New Hope', 'Saint Paul', 'Vadnais Heights', 'Richfield', 'Saint Paul', 'Shoreview']
Simulation 91 | Timestep 9: BMSB spread to 14 location(s): ['Blaine', 'Bloomington', 'Bloomington', 'Brooklyn Center', 'Eagan', 'Hopkins', 'Golden Valley', 'Golden Valley', 'Little Canada', 'Maplewood', 'Saint Louis Park', 'Plymouth', 'Savage', 'Spring Lake Park']
Simulation 91 | Timestep 10: BMSB spread to 13 location(s): ['Robbinsdale', 'Coon Rapids', 'Woodbury', 'Mendota Heights', 'Eden Prairie', 'Minnetonka', 'Lake Elmo', 'Lakeville', 'Mahtomedi', 'Minnetrista', 'Scandia', 'West Saint Paul', 'Woodbury']
Simulation 91 | Timestep 11: BMSB spread to 15 location(s): ['Apple Valley', 'Mounds View', 'Inver Grove Heights', 'Prior Lake', 'Rosemount', 'Fridley', 'Brooklyn Park', 'Chaska', 'Chisago City', 'Farmington', 'Fridley', 'Inver Grove

Simulation 95 | Timestep 24: BMSB spread to 1 location(s): ['North Oaks']
Simulation 95 | Timestep 25: BMSB spread to 1 location(s): ['Lindstrom']
Simulation 95 | Timestep 26: BMSB spread to 3 location(s): ['East Bethel', 'Newport', 'Oak Park Heights']
Simulation 95 | Timestep 27: BMSB spread to 0 location(s): []
Simulation 95 | Timestep 28: BMSB spread to 1 location(s): ['Rice Lake']
Simulation 95 | Timestep 29: BMSB spread to 4 location(s): ['Long Lake', 'Willmar', 'Montrose', 'Stewartville']
Simulation 95 Complete
Simulation 96 | Timestep 0: BMSB spread to 0 location(s): []
Simulation 96 | Timestep 1: BMSB spread to 0 location(s): []
Simulation 96 | Timestep 2: BMSB spread to 0 location(s): []
Simulation 96 | Timestep 3: BMSB spread to 0 location(s): []
Simulation 96 | Timestep 4: BMSB spread to 0 location(s): []
Simulation 96 | Timestep 5: BMSB spread to 0 location(s): []
Simulation 96 | Timestep 6: BMSB spread to 0 location(s): []
Simulation 96 | Timestep 7: BMSB spread to 0 locat